In [1]:
# 🔧 Setup & Import
import sys
import os

sys.path.append(
    os.path.join(
        os.path.abspath(os.path.join(os.getcwd(), "..")) , "src"
    )
)

from schema_linking.schema.extract_schema import extract_active_column_texts
from schema_linking.keywords.keyword_extractor import llama_keyword_extraction
from schema_linking.retrieval.retriever import build_or_load_index, retrieve_top_k
from schema_linking.llm.llama_linker import llama_table_linking, llama_column_linking, llama_filter_columns_by_keywords
from schema_linking.utils.printer import (
    print_header,
    print_keywords,
    pretty_print_list,
    print_summary
)
from schema_linking.config.config import TOP_K_COLUMNS

In [2]:
# ✍️ Input della Query
print_header("Input")
question = "Quali attività IVA hanno ancora validità nel 2024?"
print(f"Question: {question}")


Input
Question: Quali attività IVA hanno ancora validità nel 2024?


In [6]:
# 🔍 Step 1: Extracting keywords using LLM...
print_header("Step 1: Extracting keywords using LLM...")
keywords = llama_keyword_extraction(question)
print_keywords(keywords)
keyword_string = " ".join(keywords)


Step 1: Extracting keywords using LLM...


c:\Workspace\NL2SQL\venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.102.17'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



Keywords:
1. attività
2. IVA
3. validità
4. 2024


In [3]:
# 🧱 Step 2: Schema extraction
print_header("Step 2: Loading schema...")
column_texts = extract_active_column_texts()
print(f"Loaded {len(column_texts)} columns")


Step 2: Loading schema...
Filtered ba_table_fields from 47388 to 3908 fields (matching active tables).
Loaded 3908 columns


In [4]:
# 🧠 Step 3: FAISS Pruning
print_header(f"Step 3: FAISS column pruning (top {TOP_K_COLUMNS}) using keywords...")
column_index, column_map = build_or_load_index(column_texts)
pruned_columns = retrieve_top_k(column_index, column_map, question, TOP_K_COLUMNS)

if pruned_columns:
    pretty_print_list("Pruned Columns (FAISS)", pruned_columns)
else:
    print("No columns found")


Step 3: FAISS column pruning (top 50) using keywords...
Loading FAISS index from disk → c:\Workspace\NL2SQL\data\faiss_columns.bin

Pruned Columns (FAISS):
1. ei_reportvat_par.VPPARIVA: Partita IVA del contribuente (tabella: EI - Parametri per Liquidazioni)
2. ei_reportvat_par.VPPARCON: Partita IVA controllante (tabella: EI - Parametri per Liquidazioni)
3. ei_docstati.DSFLGLI: [OBSOLETO] - Indica che lo stato � attivo per la liquidazione IVA (tabella: FATEL - Stati documento)
4. ba_uffiva.UIDESCRI: Descrizione Ufficio IVA (tabella: CG - Ufficio Iva)
5. ba_codiva.CIPERIVA: % Aliquota (tabella: CG - Codici IVA)
6. ba_pagdett.PDFL_IVA: Parametro IVA (tabella: BA - Pagamenti Dettagli)
7. ei_reportvat_par.VPCODFSC: Codice fiscale della societ� (tabella: EI - Parametri per Liquidazioni)
8. ba_contact.COTIPSOG: Tipo soggetto IVA (tabella: CO - Contatto)
9. ba_codiva.CICODCOL: Codice IVA collegato (tabella: CG - Codici IVA)
10. ei_reportvat_par.VPCODFIN: Codice fiscale dell'incaricato (tabell

In [13]:
import numpy as np
from sentence_transformers import SentenceTransformer
from schema_linking.config.config import (
    EMBEDDING_MODEL,
)

model = SentenceTransformer(EMBEDDING_MODEL)

query_vec = model.encode([question], convert_to_numpy=True, normalize_embeddings=True)
D, I = column_index.search(query_vec, len(column_map))
[(i+1, D[0][i], column_map[I[0][i]]) for i in range(150) if "DTENDVAL" in column_map[I[0][i]] or "attivagen" in column_map[I[0][i]]]

[(113,
  np.float32(0.35967377),
  'ba_attivagen.AICODICE: Codice Attività Iva (tabella: CG - Tabella attività Iva)'),
 (133,
  np.float32(0.35319874),
  'ba_aspest.DTENDVAL: Data fine validit� (tabella: IS - Aspetto dei beni)'),
 (146,
  np.float32(0.349522),
  'ba_tippaga.DTENDVAL: Fine validita (tabella: BA - Tipi di Pagamento)')]

In [7]:
# 🧠 Step 4: LLM filtering on pruned columns
print_header("Step 4: LLM filtering on pruned columns...")
llm_filtered_columns = llama_filter_columns_by_keywords(keyword_string, pruned_columns, keywords)

if llm_filtered_columns:
    pretty_print_list("Filtered Columns (LLM)", llm_filtered_columns)
else:
    print("No columns passed the LLM filtering step")


Step 4: LLM filtering on pruned columns...


c:\Workspace\NL2SQL\venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.102.17'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



Filtered Columns (LLM):
1. ba_codiva.CICODCOL
2. ba_codiva.CIGRACOM
3. ba_codiva.CIGRACES
4. ei_reportvat_par.VPCODFSC
5. ei_reportvat_par.VPCODFCO
6. ei_reportvat_par.VPULMESE
7. ei_reportvat_par.VPPARIVA
8. ei_reportvat_par.VPPARCON
9. ei_regfis.RFCODICE
10. ba_codiva.CINDCINV
11. ba_codiva.CIPERIVA
12. ei_reportvat_par.VPFIRSOC
13. ei_dattra.DTIDEFIS
14. ba_values.VAARRIVA
15. ba_values.VADESCRI
16. ei_reportvat_par.VPCODFRE
17. ei_reportvat_par.VPCODICE
18. ei_as_tratfis.TFCODICE


In [7]:
# 🧮 Step 5: Table Linking via LLM
print_header("Step 5: LLM-based table linking (using filtered columns)...")
candidate_tables = list(set([col.split('.')[0] for col in llm_filtered_columns]))

linked_tables = llama_table_linking(question, candidate_tables)

if linked_tables:
    pretty_print_list("Linked Tables (LLM)", linked_tables)
else:
    print("No tables linked by LLM")


Step 5: LLM-based table linking (using filtered columns)...


c:\Workspace\NL2SQL\venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.102.17'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



Linked Tables (LLM):
1. ba_liqiva
2. ba_diceme_2020_m
3. ba_regiva
4. ba_tmpnoseqiva
5. ba_tmpcastiva
6. ba_mycomp2
7. ba_ivareg
8. ba_rifatt
9. ba_tmpregiva
10. ei_reportvat
11. ei_reportvat_par
12. ei_reportvat_m
13. ba_tmpesidif
14. ba_tmpliqiva
15. ba_tmpliqiva2


In [8]:
# 🔗 Step 6: Final Column Linking
print_header("Step 6: LLM-based final column linking...")
final_columns = [col for col in llm_filtered_columns if col.split('.')[0] in linked_tables]
linked_columns = llama_column_linking(keyword_string, final_columns)

if linked_columns:
    print_summary(question, keywords, linked_tables, linked_columns)
else:
    print("No columns linked by LLM")


Step 6: LLM-based final column linking...


c:\Workspace\NL2SQL\venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.102.17'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



Final Result
Question: Quali attività IVA risultano ancora valide nel 2024?

Keywords:
1. Attività
2. IVA
3. 2024
4. risultano
5. valida

Linked Tables (LLM):
1. ba_ivareg
2. ei_tmpcastiva
3. ba_diceme_2020_m
4. ei_reportvat
5. ba_tmpliqiva
6. ei_reportvat_m
7. ba_tmpregiva
8. ba_diceme_m

Linked Columns (LLM):
1. - ei_reportvat_m.VPPARIVA
2. - ba_tmpliqiva.LITOTIVA2
3. - ei_reportvat.VPIDEB12
4. - ei_reportvat.VPIDEB12_CS
5. - ei_reportvat.VPIMPOR6
6. - ei_reportvat.VPICRE12
7. - ba_tmpliqiva.LITOTIND
8. - ba_tmpliqiva.LITOTDET
9. - ba_tmpregiva.SRPROIND
10. - ba_tmpregiva.SRIVAIND
11. - ba_tmpliqiva.LIPROIND
12. - ba_tmpliqiva.LIPRODET
13. - ba_diceme_2020_m.DECOMPLA
14. - ba_diceme_m.DECOMPLA
